In [4]:
from datasets import load_dataset

In [5]:
# Load the MedQA test dataset
medqa_test_set = load_dataset("json",data_files="../data/MedQA/questions/US/test.jsonl")
# hint: data_files is a list of file paths.the first parameter is the dataset format

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
print(medqa_test_set['train'][:20])

{'question': ['A junior orthopaedic surgery resident is completing a carpal tunnel repair with the department chairman as the attending physician. During the case, the resident inadvertently cuts a flexor tendon. The tendon is repaired without complication. The attending tells the resident that the patient will do fine, and there is no need to report this minor complication that will not harm the patient, as he does not want to make the patient worry unnecessarily. He tells the resident to leave this complication out of the operative report. Which of the following is the correct next action for the resident to take?', "A 67-year-old man with transitional cell carcinoma of the bladder comes to the physician because of a 2-day history of ringing sensation in his ear. He received this first course of neoadjuvant chemotherapy 1 week ago. Pure tone audiometry shows a sensorineural hearing loss of 45 dB. The expected beneficial effect of the drug that caused this patient's symptoms is most l

In [7]:
#it can be seen from the dataset that the diagnosis questions suit our needs most. 
filtered_medqa_test_set = medqa_test_set['train'].filter(lambda example: 'diagnosis' in example['question'] )

Filter:   0%|          | 0/1273 [00:00<?, ? examples/s]

In [8]:
print(filtered_medqa_test_set)

Dataset({
    features: ['question', 'answer', 'options', 'meta_info', 'answer_idx'],
    num_rows: 189
})


In [9]:
#A confirmatory test is then performed in the mother to confirm the diagnosis of HIV. Which of the following is most true about the confirmatory test? This question is not good enough.we need other filtering methods.
# Which of the following is the most likely diagnosis?' is a good question.

In [10]:
filtered_medqa_test_set2 = medqa_test_set['train'].filter(lambda example: 'the most likely diagnosis' in example['question'] )

Filter:   0%|          | 0/1273 [00:00<?, ? examples/s]

In [11]:
print(filtered_medqa_test_set2)

Dataset({
    features: ['question', 'answer', 'options', 'meta_info', 'answer_idx'],
    num_rows: 125
})


In [12]:
#much better now

In [13]:
#这样子就可以了，将其转化为OSEC等格式其实并无必要

In [14]:
num_cases = 5  # number of cases to generate
#the first 5 cases for example

In [15]:
import openai
#the different between system and user：system role will exist longer than user role

In [16]:
openai.api_key =""

In [57]:
template = """ 
{ 
    "Patient Information": {
      "Gender and Age": 例如"67-year-old man",
      "Patient History":请在这里填写除了"性别年龄、Test_Results、disease、Physical_Examination_Findings"以外的所有信息,可以与其它非"性别年龄、Test_Results、disease、Physical_Examination_Findings"的栏目重复，但不能遗漏。请将最近的检查结果放置于"Test_Results"或"Physical_Examination_Findings"一栏,
      "Chief Complaint": 在这里填写患者此次就诊的主要原因或最突出的症状，如 "Right knee swelling and pain",
      "Secondary Symptoms": 在这里填写患者的次要症状，如 "Back pain",
      "Past_Medical_History":在这里填写患者以前诊断过的疾病、健康问题及相关治疗,
      "Social_History":如果存在吸烟、饮酒、药物使用、饮食习惯、运动情况。职业与居住工作环境，旅行史，家庭和人际关系，婚姻状况、文化与宗教等因素等与social_history有关的信息请填写在这里。
      "Review_of_Systems": 在这里填写其余的额外事项，包括患者否认的症状,
    },
    "Physical_Examination_Findings": {
      "Main_Signs": { 在这里填写体温、血压、心率、呼吸率等信息，如果有的话
      },
      "System Examination": {
      Physical examination is notable for pitting of his nails.
      }
    },
    "Test_Results": 在这里填写所有的检查结果和检查项目名，例如"Laboratory"、"Imaging"、"Other_Tests"等等,所有涉及到数字的实验室检查都应该放在这里。
    },
}
"""

In [58]:
print(filtered_medqa_test_set2)

Dataset({
    features: ['question', 'answer', 'options', 'meta_info', 'answer_idx'],
    num_rows: 125
})


In [59]:
# for instance in filtered_medqa_test_set3:
messages = [
    {"role": "system", "content": "你是一个用于将MedQA医疗数据集中的医疗问题转化为交互式评价基准的专家。你的目标是生成一个满足要求的结构化JSON文件。你不能遗漏任何在数据中出现的信息。"},
    {"role": "user", "content": " 请按照接下来提供的模板填充数据{}.".format(template) + "请阅读 \"answer\" 类别以获得正确的诊断结果，并将其填充在\"Correct_Diagnosis\"一栏\n\n下面是你需要修改的信息{}".format(filtered_medqa_test_set2[4]) + """\n\n请直接回复正确的填充结果\n"""}
]
# Generate OSCE json
response = openai.ChatCompletion.create(
        model="gpt-4-turbo-preview",
        messages=messages,
    )
print(response.choices[0].message['content'])

```json
{
  "Patient Information": {
    "Gender and Age": "27-year-old man",
    "Patient History": "The patient was involved in a motor vehicle collision as the front seat unrestrained driver in a head-on collision. The patient was given 2 liters of Ringer lactate solution and morphine for initial management.",
    "Chief Complaint": "Chest pain",
    "Secondary Symptoms": "Bruising over his chest wall",
    "Past_Medical_History": "Not provided",
    "Social_History": "Not provided",
    "Review_of_Systems": "The review of systems is not fully documented but includes the patient's current symptoms."
  },
  "Physical_Examination_Findings": {
    "Main_Signs": {
      "Temperature": "99.5°F (37.5°C)",
      "Blood Pressure": "107/58 mmHg",
      "Pulse": "120/min",
      "Respirations": "17/min",
      "Oxygen Saturation": "98% on room air"
    },
    "System Examination": "Physical examination is notable for an uncomfortable young man with bruising over his chest wall."
  },
  "Test_

In [60]:
print(filtered_medqa_test_set2[4])

{'question': 'A 27-year-old man presents to the emergency department after a motor vehicle collision. The patient was the front seat unrestrained driver in a head on collision. The patient’s echocardiogram (ECG) is notable only for sinus tachycardia. His temperature is 99.5°F (37.5°C), blood pressure is 107/58 mmHg, pulse is 120/min, respirations are 17/min, and oxygen saturation is 98% on room air. The patient is given 2 liters of Ringer lactate solution and morphine. Initial workup demonstrates that the patient’s pulmonary capillary wedge pressure and troponins are elevated. The patient is currently complaining of chest pain. Physical exam is notable for an uncomfortable young man with bruising over his chest wall. Which of the following is the most likely diagnosis?', 'answer': 'Cardiac contusion', 'options': {'A': 'Cardiac contusion', 'B': 'Hemorrhage', 'C': 'Myocardial infarction', 'D': 'Pulmonary contusion', 'E': 'Takotsubo cardiomyopathy'}, 'meta_info': 'step2&3', 'answer_idx': 

In [73]:
for i in range(3):
    messages = [
        {"role": "system", "content": "你是一个用于将MedQA医疗数据集中的医疗问题转化为交互式评价基准的专家。你的目标是生成一个满足要求的结构化JSON文件。你不能遗漏任何在数据中出现的信息。"},
        {"role": "user", "content": " 请按照接下来提供的模板填充数据{}.".format(template) + "请阅读 \"answer\" 类别以获得正确的诊断结果，并将其填充在\"Correct_Diagnosis\"一栏\n\n下面是你需要修改的信息{}".format(filtered_medqa_test_set2[i]) + """\n\n请直接回复正确的填充结果\n"""}
    ]
    # Generate OSCE json
    response = openai.ChatCompletion.create(
            model="gpt-4-turbo-preview",
            messages=messages,
        )
    # Save the generated json file
    with open('MedQA_organized.json', 'a', encoding='utf-8') as json_file:
        json_file.write(response.choices[0].message['content'])
    